# **Smart-UJ UAQUE**
Este Notebook hace parte del servicio inteligente "Recomendaciones por temáticas para perfil grupal" y tiene como principal objetivo construir el modelo de Recomendación basado en el contenido del material bibliográfico de la Biblioteca Alfonso Borrero Cabal. 

**Integrantes:**</br>
Johan Ortegón </br>
Juan Angarita </br>
Jorge Salgado </br>
Daniel Tibaquira </br></br>
**Directora de Proyecto de Grado:** </br>
Viky Arias </br>

# Carga de datos

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
import pandas as pd
import numpy as np

In [10]:
# Archivo donde se calcularon los pesos para cada prestamo

# Para colab
# df_prestamos = pd.read_json('/content/drive/MyDrive/UAQUE/TABLA_JOIN.json')

# Para local
df_prestamos = pd.read_json('./TABLA_JOIN.json')

df_prestamos.head()

,RowID,Mes,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,TipoItem_x,Llave_x,Perfil,...,Copias,Autor,Titulo,Tema650,AnioPublicación,NO_DEWEY,ENCABEZAMIENTO,original,Temas,union
0,Row0,Enero,1546992000000,80000005327627,720.98 A71S,720,COL-GRAL-2,LIBRO,866245,A-EGRESADO,...,1,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,arquitectura historia america latina,2012.0,720.98,Arquitectura latinoamericana,arquitectura historia america latina,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,Enero,1546992000000,80000001191496,720.9861 A71,720,COL-GRAL-2,LIBRO,309945,A-EGRESADO,...,2,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,arquitectura colonial historia colombia,1993.0,720.9861,Arquitectura colombiana,arquitectura colonial historia colombia,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,Enero,1547078400000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,LIBRO,822727,A-EGRESADO,...,2,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,quimica,2010.0,540,Química,quimica,[quimic],quimic
3,Row3,Enero,1547078400000,80000004509512,512 B151,510,COL-GRAL-3,LIBRO,784541,A-EGRESADO,...,1,"Baldor, J. Aurelio",Álgebra con gráficos y 6523 ejercicios y probl...,algebra,2009.0,512,Álgebra,algebra,[algebr],algebr
4,Row4,Enero,1547251200000,80000005270756,712 M17E,710,COL-GRAL-2,LIBRO,858473,A-EGRESADO,...,2,"Martínez de Pisón, Eduardo 1937-",Miradas sobre el paisaje Eduardo Martínez de P...,arquitectura del paisaje,2009.0,712,Arquitectura del paisaje,arquitectura del paisaje,"[arquitectur, paisaj]",arquitectur paisaj


In [102]:
# Archivo donde se calcularon los pesos para cada Llave POR CLUSTERS

# Para colab
# df = pd.read_json('/content/drive/MyDrive/UAQUE/Recomendacion/pesos_con_cluster_final.json')

# Para local
df = pd.read_json('./pesos_con_cluster_final.json')

df.head()

,Llaves,cluster,Dewey,Peso del prestamos
0,199645,69,900,0.50
1,199645,74,900,0.50
2,199800,127,Mapa,0.25
3,199810,148,Mapa,0.25
4,199825,148,Mapa,0.25


In [5]:
# Archivo donde se calcularon los pesos para cada prestamo POR USUARIOS
# df = pd.read_json('/content/drive/MyDrive/UAQUE/Recomendacion/data_to_build.json')
# df.head()

In [6]:
# Matriz de usuarios con respecto a los deweys
# users_deweys = df.pivot_table(index='ID de usuario ok', columns='Dewey', values='Peso del prestamos', aggfunc=np.sum)
# users_deweys.head()

In [7]:
# users_deweys.to_json('data_to_rec_deweys.json')

## Instalar Librería de recomendaciones (En Colab, para local instalar aparte mejor)

In [8]:
# !pip3 install surprise

In [9]:
# !pip3 install ipynb

In [10]:
# !pip3 install import-ipynb

## Funciones auxiliares

In [12]:
# Función para obtener los items ordenados según su rango, este depende del peso dado a cada ítem y de cuántas veces ha sido calificado
def getPopularityRank():
    ratings = defaultdict(int)
    rankings = defaultdict(int)
    
    for llave in df['Llaves']:
        ratings[llave] += 1

    rank = 1
    for llave, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
        rankings[llave] = rank
        rank += 1
    return rankings


In [31]:
#Para obtener el nombre de un material a partir del título
def itemIdToMaterial(id):
    return df_prestamos.loc[df_prestamos['Llave_x'] == id]['Titulo'].unique()

In [29]:

def itemIdToDewey(id):
    return (df_prestamos.loc[df_prestamos['Llave_x'] == id]['Dewey'].unique())[0]

In [15]:
# Función para obtener el diccionario de Deweys
import json
from collections import defaultdict

def getAllDeweys():
    with open('./deweys.json') as json_file:
        data = json.load(json_file)
    return defaultdict(list, data)

In [16]:
# Función para obtener el diccionario de Años de publicación
import json
from collections import defaultdict

def getAllPublishedYears():
    with open('./years.json') as json_file:
        data = json.load(json_file)
    return defaultdict(int, data)

In [17]:
# Función para obtener el diccionario de temáticas
import json
from collections import defaultdict

def getAllThemes():
    with open('./themes.json') as json_file:
        data = json.load(json_file)
    return defaultdict(list, data)

In [18]:
def loadSimilarities():
    return np.load('similarities_rec.npy')

In [19]:
import sys
sys.path.append('./')
#!cp -r "/content/drive/MyDrive/UAQUE/Recomendacion/model_prep_train_evaluate.py" '/content/'

In [20]:
import model_prep_train_evaluate as cbrec

# Construcción del Modelo

In [20]:
# Para calcular las matrices de similaridad
# genres = getAllThemes()
# years = getAllPublishedYears()

In [21]:
from surprise import AlgoBase
from surprise import PredictionImpossible
import math
import numpy as np
import heapq


class ContentKNNAlgorithm(AlgoBase):

    def __init__(self, k=40, sim_options={}):
        AlgoBase.__init__(self)
        self.k = k

    def fit(self, trainset):
        print("Pre fit base")
        AlgoBase.fit(self, trainset)
        print("Post fit base")
        # Calculando la matrix de similaridad entre ítems basado en su contenido
        # En este caso solo genres = temáticas y years = Año de publicación
        # genres = getAllThemes()
        # years = getAllPublishedYears()
        
        print("Calculando matriz de similaridad basada en contenido...")
        # Calcula la distancia entre Dewys para cada combinación de ítems como una matriz 2x2
        simsalgo = evaluator.dataset.GetSimMatrix()
        self.similarities = simsalgo

#         for thisRating in range(self.trainset.n_items):
#             if (thisRating % 100 == 0):
#                 print(thisRating, " de ", self.trainset.n_items)
#             for otherRating in range(thisRating+1, self.trainset.n_items):
#                 thisItemID = int(self.trainset.to_raw_iid(thisRating))
#                 print("thisItemID", thisItemID)
#                 otherItemID = int(self.trainset.to_raw_iid(otherRating))
#                 print("otherItemID", otherItemID)
#                 genreSimilarity = self.computeGenreSimilarity(thisItemID, otherItemID, genres)
#                 yearSimilarity = self.computeYearSimilarity(thisItemID, otherItemID, years)
#                 self.similarities[thisRating, otherRating] = genreSimilarity * yearSimilarity
#                 self.similarities[otherRating, thisRating] = self.similarities[thisRating, otherRating]
                
        print("...done.")
                
        return self
    
    # Calculo de similitud de temáticas

    
    # Calculo de similitud de año de publicación
    def computeYearSimilarity(self, item1, item2, years):
        diff = abs(years[str(float(item1))] - years[str(float(item2))])
        sim = math.exp(-diff / 10.0)
        return sim

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('Usario o item desconocidos')
        
        # Construimos los puntajes entre el ítem y las pesos dados a los temas
        neighbors = []
        for rating in self.trainset.ur[u]:
            genreSimilarity = self.similarities[i,rating[0]]
            neighbors.append( (genreSimilarity, rating[1]) )
        
        # Extraer los top-K ratings más similares
        k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda t: t[0])
        
        # Calculo promedio de similitud de los K vecinos con respecto a los ratings del usuario
        simTotal = weightedSum = 0
        for (simScore, rating) in k_neighbors:
            if (simScore > 0):
                simTotal += simScore
                weightedSum += simScore * rating
            
        if (simTotal == 0):
            raise PredictionImpossible('No tiene vecinos')

        predictedRating = weightedSum / simTotal

        return predictedRating
    


In [22]:
# def fitComputeUtil():


# Predict

Esta función permite pasar el dataset de pesos a un Database de Surprise para poder hacer uso de la librería de recomendaciones

In [21]:
from numpy import interp
from sklearn.utils import shuffle
dfP = (df[0:80000])
#dfP = df
scale = 5

toSurprisedb = pd.DataFrame(columns=['userID', 'itemID', 'rating'])

# Para USUARIOS
# toSurprisedb['userID'] = dfP['ID de usuario ok']

# Para CLUSTER
toSurprisedb['userID'] = dfP['cluster']

toSurprisedb['itemID'] = dfP['Llaves']

maxRat = dfP['Peso del prestamos'].max()
mappedWeights = dfP['Peso del prestamos'].map(lambda x: interp(x,[0, maxRat],[0, scale]))
toSurprisedb['rating'] = mappedWeights

In [24]:
#toSurprisedb.to_json('trainset_to_rec.json')

In [25]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
import random
import numpy as np

np.random.seed(0)
random.seed(0)

reader = Reader(rating_scale=(0, scale))

# Utilizamos las funciones definidas en el notebook de preparación y evaluación 
# Para generar los set de datos a evaluar
evaluate = Dataset.load_from_df(toSurprisedb[['userID', 'itemID', 'rating']], reader)

rankings = getPopularityRank()

# Este se encargará de entrenar y generar las métricas
evaluator = cbrec.Evaluator(evaluate, rankings)

contentKNN = ContentKNNAlgorithm()
evaluator.AddAlgorithm(contentKNN, "ContentKNN")

# Modelo Random para comparar
# Random = NormalPredictor()
# evaluator.AddAlgorithm(Random, "Random")


trainSet = evaluator.dataset.trainSet

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [26]:
# Entrenamos y evaluamos todos los modelos agregados
evaluator.Evaluate(True)


Evaluando el algoritmo  ContentKNN ...
Calculando Precisión...
Pre fit base
Post fit base
Calculando matriz de similaridad basada en contenido...
...done.
Evaluando los top-10 utilizando Leave One Out...
Pre fit base
Post fit base
Calculando matriz de similaridad basada en contenido...
...done.
Computing hit-rate and rank metrics...
Calculando recomiendaciones para todo el data set...
Pre fit base
Post fit base
Calculando matriz de similaridad basada en contenido...
...done.
Analizando cobertura, diversidad y novedad...
Análisis Completado


Algoritmo  RMSE       MAE        HR         cHR        ARHR       Cobertura  Diversidad Novedad   
ContentKNN 0.1243     0.0720     0.0000     0.0000     0.0000     0.0000     0.0000     1.0000    

Resumen:

RMSE:      Root Mean Squared Error. Valores más bajos significa mejor precisión.
MAE:       Mean Absolute Error. Valores más bajos significa mejor precisión.
HR:        Hit Rate; Qué tan seguido recomendamos una predicción Left-Out (fuera del 

Generamos las recomendaciones para un usuario en específico

In [27]:
def GetAntiTestSetForUser(testSubject):
    trainset = trainSet
    fill = trainset.global_mean
    anti_testset = []
    u = trainset.to_inner_uid((testSubject))
    user_items = set([j for (j, _) in trainset.ur[u]])
    anti_testset += [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), fill) for
                                i in trainset.all_items() if
                                i not in user_items]
    return anti_testset

In [28]:
def SampleTopNRecs(testSubject=92, k=10):
    print(len(list(set(evaluator.algorithms))))
   
    for algo in set(evaluator.algorithms):
        print("\nUsando el recomendador: ", algo.GetName())
        
        # print("\nConstruyendo el modelo de recomendación...")
        # # trainSet = self.dataset.GetFullTrainSet()
        # algo.GetAlgorithm().fit(trainSet)
        
        print("Computing recommendations...")
        testSet = GetAntiTestSetForUser(testSubject)
    
        predictions = algo.GetAlgorithm().test(testSet)
        
        recommendations = []
        
        for userID, movieID, actualRating, estimatedRating, _ in predictions:
            intMovieID = int(movieID)
            recommendations.append((intMovieID, estimatedRating))
        
        recommendations.sort(key=lambda x: x[1], reverse=True)
        
        data_recs = []
        for ratings in recommendations[:10]:
            data_recs.append({'Titulo':itemIdToMaterial(ratings[0]), 
                             'Rating':ratings[1], 
                             'Dewey':itemIdToDewey(ratings[0]), 
                             'Tematicas':itemIdToThemes(ratings[0])})
        
        return pd.DataFrame(data_recs)


In [33]:
df_temas = pd.read_json('./Libros.json')

In [34]:
def itemIdToThemes(id):
    tematicas = df_temas.loc[df_temas['Llaves'] == str(id)]['Tema 670'].values
    if len(tematicas) == 0:
        return []
    else:
        return tematicas

In [40]:
dfP['cluster'].value_counts()

34     5080
95     3925
93     3577
30     2658
148    2253
       ... 
100      39
57       38
74       37
78       28
32        4
Name: cluster, Length: 150, dtype: int64

In [6]:
eval_cluster = 0

In [7]:
df_recs = SampleTopNRecs(testSubject=eval_cluster)
df_recs

NameError: name 'SampleTopNRecs' is not defined

In [43]:
user_p = dfP.loc[dfP['cluster'] == eval_cluster]
tematicas =[]
for llave in user_p['Llaves'].unique():
    tematicas.append(itemIdToThemes((llave)))

user_p['Tematicas'] = tematicas
user_p

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Llaves,cluster,Dewey,Peso del prestamos,Tematicas
223,204751,78,Diapositiva,0.250000,[]
5310,272497,78,190,3.500000,"[[filosofía, francés]]"
7598,281688,78,120,1.500000,[[ontología]]
7712,282206,78,100,0.500000,[[estética]]
13656,313615,78,400,0.250000,"[[lingüístico, investigación]]"
23204,353253,78,120,0.500000,"[[filosofía, mente]]"
28990,525552,78,770,2.000000,[[fotografía]]
29612,530818,78,120,0.250000,[[hermenéutico]]
30496,534169,78,300,0.500000,"[[civilización, occidental]]"
30720,534629,78,120,0.375000,"[[relación, sexual]]"


In [44]:
(dfP.loc[dfP['cluster'] == eval_cluster]['Dewey'].unique())

array(['Diapositiva', '190', '120', '100', '400', '770', '300', '900',
       '860', '290'], dtype=object)

In [34]:
# # Export del modelo 
# import surprise
# for algo in set(evaluator.algorithms):
#     print(algo.algorithm)
#     surprise.dump.dump('content_based_rec_model_80k', algo=algo.algorithm)


In [61]:
import gzip
import pickle
model_path = './recommenders/examples/00_quick_start/sar_trained_model.pkl'

with gzip.open(model_path, 'rb') as f:
    p = pickle.Unpickler(f)
    sar = p.load()
predictor = sar


In [95]:
eval_asr = 90

In [96]:
preds = predictor.recommend_k_items(pd.DataFrame(dict(userID=[eval_asr])), remove_seen=True) 
preds['itemID'].values

array([272388, 919253, 870716, 806209, 887461, 802276, 859932, 768666,
       539790, 361510])

In [97]:
# sims = evaluator.dataset.GetSimilarities()
# sims

In [98]:
recs = preds['itemID'].values
data_recs_asr = []
for ids in recs:
    data_recs_asr.append({'Titulo':itemIdToMaterial(ids), 
#                      'Rating':ratings[1], 
                     #'Dewey':itemIdToDewey(ids), 
                     'Tematicas':itemIdToThemes(ids)})
    
df_recs_asr = pd.DataFrame(data_recs_asr)

In [99]:
df_recs_asr

,Titulo,Tematicas
0,[Sobre la cuádruple raíz del principio de razó...,"[[filosofía, alemán]]"
1,[Crisis y salida de la crisis orden y desorden...,"[[economía, historio]]"
2,"[Nuevas minorías, nuevos derechos notas sobre ...","[[diseño, ingeniería]]"
3,[¿Es la escuela el problema? perspectivas soci...,"[[sociología, educación]]"
4,[COBITª 5 enabling processes Information Syste...,"[[cobit, administración, estándar, tecnología,..."
5,[Alla prima everything I know about painting [...,"[[pintura, técnico]]"
6,[Chopin y las sílfides Montse Sanuy ; proyecto...,"[[literatura, infantil]]"
7,[El iris salvaje Louise Elisabeth Glück ; trad...,"[[poesía, estadounidense]]"
8,[Política Aristoteles ; introducción Carlos Ga...,[[]]
9,[Homo ludens Johan Huizinga ; prólogo de Domin...,"[[antropología, juego]]"


In [100]:
(dfP.loc[dfP['cluster'] == eval_asr]['Dewey'].unique())

array(['800', '510', '460', '930', '860', '370', '140', '920', '750',
       '220', '240', '190', '500', '70', '980', '300', '820', '120',
       '830', '340', '650'], dtype=object)

In [101]:
user_p = dfP.loc[dfP['cluster'] == eval_asr]
tematicas =[]
for llave in user_p['Llaves'].unique():
    tematicas.append(itemIdToThemes((llave)))

user_p['Tematicas'] = tematicas
user_p

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Llaves,cluster,Dewey,Peso del prestamos,Tematicas
1509,256364,90,800,0.25,"[[filosofía, literatura]]"
4180,266771,90,510,0.50,"[[categoria, matemático]]"
6019,275790,90,460,0.25,"[[español, pronunciación]]"
6054,275973,90,460,0.25,"[[español, gramatica]]"
6417,277349,90,800,0.25,"[[semiótico, literatura]]"
...,...,...,...,...,...
78088,834303,90,800,0.50,"[[literatura, historia, crítica]]"
79107,840622,90,800,0.50,"[[redacción, escrito, técnico]]"
79875,842740,90,800,1.00,"[[cuento, ciencia, ficción, colección]]"
79955,842912,90,510,0.50,"[[teoría, modelo]]"
